In [1]:
import pandas as pd

data = pd.read_csv('data/^NDX_raw_data.csv')
data.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

data_backup = data.iloc[3524:]

data = data.iloc[:3524]
data_copy = data.copy()

print('Data imported and copied.', flush=True)

Data imported and copied.


In [2]:
import numpy as np

## Creating sequences
def create_dataset(dataset, time_step=1, output_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-output_step):
        a = dataset[i:(i+time_step), 0]
        b = dataset[(i+time_step):(i+time_step)+output_step, 0]
        dataX.append(a)
        dataY.append(b)

    return np.array(dataX), np.array(dataY)

period = 60
trend_period = 14
rsi_period = 14
num_features = 3
input_period = 46
output_step = 7
units = 512

In [3]:
X, y = create_dataset(data[['close']].to_numpy(), time_step=period, output_step=output_step)

In [4]:
data_input = np.array([np.concatenate((X[i], y[i]), axis=0) for i in range(X.shape[0])])

In [5]:
from statsmodels.tsa.seasonal import seasonal_decompose

decompositions = np.array([seasonal_decompose(data_input[i], model='additive', period=14) for i in range(data_input.shape[0])])
trends = np.array([decompositions[i].trend for i in range(decompositions.shape[0])])
seasons = np.array([decompositions[i].seasonal for i in range(decompositions.shape[0])])

In [6]:
from talib import RSI

rsi = np.array([ RSI(data_input[i]) for i in range(data_input.shape[0]) ])

In [7]:
trends_dropna = list()
seasons_cropped = list()
rsi_dropna = list()

for trend in trends:
    trends_dropna.append(trend[~np.isnan(trend)])

for season in seasons:
    #seasons_cropped.append(season[int(trend_period/2):-int(trend_period/2)])
    seasons_cropped.append(season[trend_period:])

for r in rsi:
    #rsi_dropna.append(r[-53:])
    rsi_dropna.append(r[rsi_period:])

trends_dropna = np.array(trends_dropna)
seasons_cropped = np.array(seasons_cropped)
rsi_dropna = np.array(rsi_dropna)

In [8]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = list(MinMaxScaler() for i in range(trends_dropna.shape[0]))
trends_scaled = list()

scaler_seasonal = list(MinMaxScaler() for i in range(seasons_cropped.shape[0]))
seasons_scaled = list()

scaler_rsi = list(MinMaxScaler() for i in range(rsi_dropna.shape[0]))
rsi_scaled = list()

for i in range(trends_dropna.shape[0]):
    trends_scaled.append(scaler[i].fit_transform(trends_dropna[i].reshape(-1,1)))

for i in range(seasons_cropped.shape[0]):
    seasons_scaled.append(scaler_seasonal[i].fit_transform(seasons_cropped[i].reshape(-1,1)))

for i in range(rsi_dropna.shape[0]):
    rsi_scaled.append(scaler_rsi[i].fit_transform(rsi_dropna[i].reshape(-1,1)))

trends_scaled = np.array(trends_scaled)
seasons_scaled = np.array(seasons_scaled)
rsi_scaled = np.array(rsi_scaled)

In [9]:
X_input = list()
y_input = list()

for trend, season, rsi in zip(trends_scaled, seasons_scaled, rsi_scaled):
    X_input.append(np.hstack((trend[:trend.shape[0]-7], season[:season.shape[0]-7], rsi[:rsi.shape[0]-7])))
    y_input.append(trend[trend.shape[0]-7:])

X_input = np.array(X_input)
y_input = np.array(y_input)

In [10]:
X_input.shape

(3457, 46, 3)

# **Main Model**

In [11]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Conv1D, AveragePooling1D, Flatten, Reshape, SimpleRNN, GRU, MaxPooling1D, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.initializers import Zeros

inputs = Input(shape=(input_period, num_features))

model_cnn = Sequential([
    Conv1D(filters=512, kernel_size=1, activation='relu'),
    Conv1D(filters=512, kernel_size=1, activation='relu'),
    MaxPooling1D(pool_size=3),
    Dense(units=128),
    Flatten()
    #Dense(units=output_step),
    #Reshape((output_step,1))
])

model_bilstm = Sequential([
    Bidirectional(LSTM(units=512, return_sequences=True, activation='tanh', recurrent_activation='sigmoid')),
    Dropout(0.2),
    Flatten()
    #Dense(units=output_step),
    #Reshape((output_step,1))
])

model_bigru = Sequential([
    Bidirectional(GRU(units=512, activation='tanh', return_sequences=True)),
    Dropout(0.4),
    Flatten()
])

model_multilayer_lstm = Sequential([
    LSTM(units=512, return_sequences=True, activation='tanh', recurrent_activation='sigmoid'),
    Dropout(0.2),
    LSTM(units=512, return_sequences=True, activation='tanh', recurrent_activation='sigmoid'),
    Flatten()
])

2024-05-27 20:31:48.663547: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 20:31:49.331184: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 20:31:49.331335: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 20:31:49.404940: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-27 20:31:49.551623: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 20:31:52.119895: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [12]:
output_cnn = model_cnn(inputs)
output_bilstm = model_bilstm(inputs)
output_bigru = model_bigru(inputs)
output_multilayer_lstm = model_multilayer_lstm(inputs)

concatenated_outputs = concatenate([output_cnn, output_bilstm, output_bigru, output_multilayer_lstm])

main_model = Sequential([
    Input(shape=(concatenated_outputs.shape[1],)),
    Dense(units=output_step),
    Reshape((output_step,1))
])

final_output = main_model(concatenated_outputs)

functional_pipeline = Model(inputs=inputs, outputs=final_output)

In [13]:
X_input.shape

(3457, 46, 3)

In [ ]:
# Compile the pipeline model
functional_pipeline.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

checkpoint = ModelCheckpoint(filepath='model_weights_6_3_main_1024/model_weights_epoch_{epoch:02d}.h5', 
                            save_best_only=True, save_weights_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the pipeline model
history = functional_pipeline.fit(X_input, y_input, epochs=150, batch_size=64, validation_split=0.2, callbacks=[checkpoint, early_stopping])

Epoch 1/150


2024-05-27 20:32:14.691662: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12058624 exceeds 10% of free system memory.
2024-05-27 20:32:14.717667: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12058624 exceeds 10% of free system memory.
2024-05-27 20:32:15.539601: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12058624 exceeds 10% of free system memory.
2024-05-27 20:32:15.564975: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12058624 exceeds 10% of free system memory.
2024-05-27 20:32:15.751098: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30638080 exceeds 10% of free system memory.


44/44 [==============================] - 215s 5s/step - loss: 0.2352 - accuracy: 0.0762 - val_loss: 0.0226 - val_accuracy: 0.0840
Epoch 2/150
 1/44 [..............................] - ETA: 3:15 - loss: 0.0252 - accuracy: 0.0848

In [ ]:
# Select the epoch
best_epoch = np.argmin(history.history['val_loss'])
best_epoch = best_epoch+1
print(f'best result is for epoch number {best_epoch}')

# Load the weights of the model at the chosen epoch
functional_pipeline.load_weights(f'model_weights_6_3_main_1024/model_weights_epoch_{best_epoch:02d}.h5')
print('Weigths for the best epoch has been loaded.')

In [ ]:
best_epoch_backup = 35

# Load the weights of the model at the chosen epoch
functional_pipeline.load_weights(f'model_weights_6_3_main_1024/model_weights_epoch_{best_epoch_backup:02d}.h5')
print('Backup: Weigths for the best epoch has been loaded.')